In [2]:
! pip install -U spacy -q

In [4]:

!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.1.123+-x86_64-with-glibc2.35
Python version   3.12.11                       
Pipelines        en_core_web_sm (3.8.0)        



In [5]:

!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:

# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [7]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [9]:
import json
f = open('/content/annotations.json')
TRAIN_DATA = json.load(f)

In [14]:
TRAIN_DATA

{'classes': ['VALUE',
  'ORG',
  'DATES',
  'QUANTITY',
  'MONEY',
  'ECON_TERM',
  'PRODUCT/ASSET'],
 'annotations': [['Shares of NMDC, India’s largest iron ore producer, are likely to be in focus on Tuesday, September 2, following the release of its August production and sales figures. Iron ore sales rose to 3.39 million tonnes (MT), compared with 3.14 MT in the same month last year.',
   {'entities': [[10, 14, 'NMDC'],
     [80, 87, 'TUESDAY'],
     [99, 100, 'VALUE'],
     [131, 137, 'AUGUST'],
     [191, 195, 'VALUE'],
     [231, 235, 'VALUE']]}],
  ['', {'entities': []}],
  ['Cumulatively, sales for April–August 2025 stood at 18.37 MT, up from 16.27 MT a year earlier. On the production side, output in August was 3.37 MT, higher than 3.07 MT in August 2024. Cumulative production for April–August 2025 reached 18.45 MT, compared with 14.43 MT in the corresponding period last year, according to the company’s regulatory filing.',
   {'entities': [[24, 41, 'DATES'],
     [128, 134, 'DA

In [10]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 15/15 [00:00<00:00, 390.30it/s]


In [11]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy


ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     25.86    0.00    0.00    0.00    0.00
 28     200        859.45   1975.24  100.00  100.00  100.00    1.00
 70     400          2.99      4.67  100.00  100.00  100.00    1.00
121     600          0.00      0.00  100.00  100.00  100.00    1.00
185     800          2.32      4.56  100.00  100.00  100.00    1.00
263    1000          0.00      0.00  100.00  100.00  100.00    1.00
363    1200          0.00      0.00  100.00  100.00  100.00    1.00
463    1400          0.00      0.00  100.00  100.00  100.00    1.00
633    1600          0.00      0.00 

In [13]:
nlp_ner = spacy.load("/content/model-best")

In [15]:
# sample input
doc = nlp_ner('''Shares of NMDC, India’s largest iron ore producer, are likely to be in focus on Tuesday, September 2, following the release of its August production and sales figures. Iron ore sales rose to 3.39 million tonnes (MT), compared with 3.14 MT in the same month last year.

Cumulatively, sales for April–August 2025 stood at 18.37 MT, up from 16.27 MT a year earlier. On the production side, output in August was 3.37 MT, higher than 3.07 MT in August 2024. Cumulative production for April–August 2025 reached 18.45 MT, compared with 14.43 MT in the corresponding period last year, according to the company’s regulatory filing.
Analysts expect NMDC’s production to reach 50 MT by FY26
In 1QFY26, volume growth was sluggish due to the early onset of the monsoon. However, steady price hikes by the company helped offset the impact of weaker volumes, resulting in healthy operating profit.
Iron ore production during the quarter stood at 12 MT (+31% YoY, -10% QoQ), while sales came in at 11.5 MT (+14% YoY, -9% QoQ). The average selling price (ASP) for the quarter rose to ₹5,851 per tonne (+9% YoY, +6% QoQ) as iron ore prices remained elevated.

Looking ahead, domestic brokerage Motilal Oswal expects volumes to rise steadily to 50 MT in FY26 and 55 MT in FY27, supported by an increase in EC limits. The brokerage has largely maintained its estimates for FY26–27, driven by stable realizations and healthy, volume-led operating growth.''')

In [16]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter